## Monarch Initiative
Accelerating precision medicine through Open Data Science

### Download all-associations database
see https://data.monarchinitiative.org/latest/tsv/index.html README file

In [20]:
import requests
from bs4 import BeautifulSoup
import os

# 目标网页地址
base_url = "https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/"
response = requests.get(base_url)

# 检查是否请求成功
if response.status_code != 200:
    print(f"❌ 页面访问失败：{response.status_code}")
else:
    # 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 提取所有 href 结尾为 .tsv.gz 的链接
    file_links = [
        a['href'] for a in soup.find_all('a', href=True)
        if a['href'].endswith('.tsv.gz')
    ]

    print(f"🔗 发现 {len(file_links)} 个文件链接")

    # 创建保存目录
    download_dir = "all_associations_files"
    os.makedirs(download_dir, exist_ok=True)

    # 下载所有文件
    for file_name in file_links:
        file_url = file_name if file_name.startswith("http") else base_url + file_name
        local_path = os.path.join(download_dir, os.path.basename(file_name))
        print(f"⬇️ 正在下载: {file_name}")
        try:
            r = requests.get(file_url, stream=True)
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        except Exception as e:
            print(f"❌ 下载失败 {file_name}: {e}")



🔗 发现 22 个文件链接
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/causal_gene_to_disease_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/chemical_to_disease_or_phenotypic_feature_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/chemical_to_pathway_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/correlated_gene_to_disease_association.all.tsv.gz
⬇️ 正在下载: https://data.monarchinitiative.org/monarch-kg/2025-04-15/tsv/all_associations/disease_or_phenotypic_feature_to_genetic_inheritance_association

In [21]:
import gzip
import shutil
import os

# 定义输入目录和输出目录
input_dir = "all_associations_files"
output_dir = "all_associations_unzipped"
os.makedirs(output_dir, exist_ok=True)

# 遍历 input_dir 中所有 .gz 文件
for filename in os.listdir(input_dir):
    if filename.endswith(".tsv.gz"):
        input_path = os.path.join(input_dir, filename)
        output_filename = filename.replace(".gz", "")  # 移除.gz后缀
        output_path = os.path.join(output_dir, output_filename)

        print(f"🧩 解压中: {filename} -> {output_filename}")
        try:
            with gzip.open(input_path, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        except Exception as e:
            print(f"❌ 解压失败 {filename}: {e}")

print("✅ 全部解压完成！")


🧩 解压中: causal_gene_to_disease_association.all.tsv.gz -> causal_gene_to_disease_association.all.tsv
🧩 解压中: disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv.gz -> disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv
🧩 解压中: genotype_to_disease_association.all.tsv.gz -> genotype_to_disease_association.all.tsv
🧩 解压中: chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv.gz -> chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv
🧩 解压中: association.all.tsv.gz -> association.all.tsv
🧩 解压中: gene_to_pathway_association.all.tsv.gz -> gene_to_pathway_association.all.tsv
🧩 解压中: macromolecular_machine_to_molecular_activity_association.all.tsv.gz -> macromolecular_machine_to_molecular_activity_association.all.tsv
🧩 解压中: correlated_gene_to_disease_association.all.tsv.gz -> correlated_gene_to_disease_association.all.tsv
🧩 解压中: disease_to_phenotypic_feature_association.all.tsv.gz -> disease_to_phenotyp

### Merge all asscociations data

In [22]:
import pandas as pd
import os

# 解压后的文件夹
input_dir = "all_associations_unzipped"
# 输出总文件名
output_file = "merged_associations_all.tsv"

# 存放所有 DataFrame
df_list = []

# 遍历文件夹下所有 .tsv 文件
for filename in os.listdir(input_dir):
    if filename.endswith(".tsv"):
        file_path = os.path.join(input_dir, filename)
        print(f"📥 正在读取: {filename}")
        try:
            # 读取每个文件
            df = pd.read_csv(file_path, sep='\t', low_memory=False)
            df_list.append(df)
        except Exception as e:
            print(f"❌ 读取失败 {filename}: {e}")

# 合并所有 DataFrame
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    # 保存为一个新的 TSV 文件
    merged_df.to_csv(output_file, sep='\t', index=False)
    print(f"✅ 所有文件已合并并保存为: {output_file}")
else:
    print("⚠️ 没有找到可以合并的文件。")


📥 正在读取: gene_to_expression_site_association.all.tsv
📥 正在读取: macromolecular_machine_to_biological_process_association.all.tsv
📥 正在读取: chemical_to_pathway_association.all.tsv
📥 正在读取: correlated_gene_to_disease_association.all.tsv
📥 正在读取: variant_to_phenotypic_feature_association.all.tsv
📥 正在读取: genotype_to_phenotypic_feature_association.all.tsv
📥 正在读取: variant_to_disease_association.all.tsv
📥 正在读取: disease_or_phenotypic_feature_to_genetic_inheritance_association.all.tsv
📥 正在读取: causal_gene_to_disease_association.all.tsv
📥 正在读取: chemical_or_drug_or_treatment_to_disease_or_phenotypic_feature_association.all.tsv
📥 正在读取: disease_or_phenotypic_feature_to_location_association.all.tsv
📥 正在读取: gene_to_gene_homology_association.all.tsv
📥 正在读取: pairwise_gene_to_gene_interaction.all.tsv
📥 正在读取: disease_to_phenotypic_feature_association.all.tsv
📥 正在读取: chemical_to_disease_or_phenotypic_feature_association.all.tsv
📥 正在读取: variant_to_gene_association.all.tsv
📥 正在读取: macromolecular_machine_to_molecular

### Reformat the data files as the BioMedGPS format

In [166]:
import pandas as pd

df = pd.read_csv("merged_associations_all.tsv", sep="\t", low_memory=False)
df.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN


In [167]:
# 检查每一类实体数据来源数据库及对应ID号
# 遍历每个唯一的 subject_category
for stype in df["subject_category"].unique():
    # 筛选该 source_category 的所有行
    sub_df = df[df["subject_category"] == stype]
    
    # 提取 subject 列冒号前缀（例如 MGI:12345 -> MGI）
    prefixes = sub_df["subject"].str.split(":").str[0].unique()
    
    # 打印结果
    print(f"{stype}: {prefixes}")

biolink:Gene: ['MGI' 'NCBIGene' 'HGNC' 'FB' 'ZFIN' 'WB' 'SGD' 'RGD' 'PomBase'
 'dictyBase' 'Xenbase']
biolink:Protein: ['PR']
biolink:ChemicalEntity: ['CHEBI']
biolink:SequenceVariant: ['CLINVAR' 'CAID']
biolink:Genotype: ['MGI' 'RGD' 'ZFIN']
biolink:Disease: ['MONDO' 'MPATH']
biolink:NamedThing: ['MAXO' 'WBLS' 'BFO' 'BSPO' 'FBcv' 'FBdv' 'MGPO' 'OBA' 'PATO' 'PHIPO' 'RO'
 'DDANAT']
biolink:PhenotypicFeature: ['UPHENO' 'WBPhenotype' 'XPO' 'FYPO' 'HP' 'MP' 'NBO' 'ZP' 'DDPHENO']
biolink:AnatomicalEntity: ['WBbt' 'XAO' 'ZFA' 'FBbt' 'UBERON' 'EMAPA']
biolink:BiologicalProcess: ['GO']
biolink:MolecularActivity: ['GO']
biolink:CellularComponent: ['GO']
biolink:LifeStage: ['HSAPDV' 'ZFS']
biolink:OrganismTaxon: ['NCBITaxon']
biolink:MolecularEntity: ['SO' 'CHR']
biolink:Cell: ['CL']


In [168]:
# 遍历每个唯一的 object_category
for stype in df["object_category"].unique():
    # 筛选该 object_category 的所有行
    object_df = df[df["object_category"] == stype]
    
    # 提取 object 列冒号前缀（例如 MGI:12345 -> MGI）
    object_prefixes = object_df["object"].str.split(":").str[0].unique()
    
    # 打印结果
    print(f"{stype}: {object_prefixes}")

biolink:AnatomicalEntity: ['EMAPA' 'UBERON' 'FBbt' 'ZFA' 'WBbt' 'XAO']
biolink:Cell: ['CL']
biolink:CellularComponent: ['GO']
biolink:BiologicalProcess: ['GO']
biolink:Pathway: ['Reactome']
biolink:Disease: ['MONDO' 'MPATH']
biolink:PhenotypicFeature: ['HP' 'MP' 'ZP' 'FYPO' 'WBPhenotype' 'XPO' 'DDPHENO' 'UPHENO' 'NBO']
biolink:Gene: ['PomBase' 'dictyBase' 'NCBIGene' 'Xenbase' 'ZFIN' 'WB' 'FB' 'MGI' 'RGD'
 'SGD' 'HGNC']
biolink:MolecularActivity: ['GO']
biolink:NamedThing: ['OBA' 'BFO' 'GO' 'WBLS' 'BSPO' 'PATO' 'FBcv' 'FBdv' 'DDANAT' 'MAXO' 'CLM']
biolink:ChemicalEntity: ['CHEBI']
biolink:LifeStage: ['ZFS' 'HSAPDV']
biolink:MolecularEntity: ['SO' 'CHR']
biolink:Protein: ['PR']
biolink:OrganismTaxon: ['NCBITaxon']


In [169]:
df_mapping = df
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN


#### MGI → Entrez ID 映射

In [170]:
# 读取 Excel 或 TSV 映射文件（请根据实际文件路径和格式选择）
mouse_mapping_df = pd.read_csv("/Users/zhuzhixing/KG/biomedgps-data/graph_data/mapping/human_mouse_gene_mappings.tsv", sep="\t") # 或 pd.read_csv("your_mapping.tsv", sep="\t")

# 构造映射字典（确保列名对应）
mgi_to_entrez = dict(zip(mouse_mapping_df["mgi_id_mouse"], mouse_mapping_df["entrez_id_mouse"]))

# 初始化 subject_mapping 列为原始 subject
df_mapping["subject_mapping"] = df_mapping["subject"]

# 替换 MGI 开头的行
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].apply(
    lambda x: mgi_to_entrez.get(x, x) if isinstance(x, str) and x.startswith("MGI:") else x
)

In [171]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410


#### CHEBI ID → DrugBank ID 映射（基于 xrefs）

In [172]:
# 1. 读取原始实体文件（请根据实际路径和分隔符修改）
entities_df = pd.read_csv("/Users/zhuzhixing/KG/biomedgps-data/graph_data/entities.tsv", sep="\t", low_memory=False)  # 或 pd.read_excel("entities.xlsx")

# 2. 提取 CHEBI ID（支持 xrefs 中包含多个以 | 分隔的标识）
def extract_chebi(xrefs):
    if pd.isna(xrefs):
        return None
    matches = re.findall(r"CHEBI:\d+", xrefs)
    return matches[0] if matches else None

entities_df["chebi_id_extracted"] = entities_df["xrefs"].apply(extract_chebi)

# 3. 保留 chebi_id_extracted 非空的行
chebi_only_df = entities_df[entities_df["chebi_id_extracted"].notna()]

# 4. 构建 CHEBI → DrugBank 映射字典
chebi_to_drugbank = dict(zip(chebi_only_df["chebi_id_extracted"], chebi_only_df["id"]))

# 5. 替换 chebi_input_df["id"] 中的 CHEBI ID 为 DrugBank ID（如果在映射表中）
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].apply(lambda x: chebi_to_drugbank.get(x, x))


In [173]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,qualifiers,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,MMO:0000658,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410


#### subject_label → DrugBank（基于 name_clean 映射）

In [174]:
# 筛选 CHEBI 类型的 compound 实体
compound_df = entities_df[entities_df["label"] == "Compound"].copy()

# 使用 compound_df，处理名称为映射键
compound_df["name_clean"] = compound_df["name"].str.strip().str.lower()

# 构建 name → DrugBank ID 映射字典
name_to_drugbank = dict(zip(compound_df["name_clean"], compound_df["id"]))

# 初始化 subject_label_mapping 列
df_mapping["subject_label_mapping"] = None

# 对 ChemicalEntity 类型的数据进行 name 匹配映射
mask_chemical = df["subject_category"] == "biolink:ChemicalEntity"
df_mapping.loc[mask_chemical, "subject_label_clean"] = df_mapping.loc[mask_chemical, "subject_label"].str.strip().str.lower()
df_mapping.loc[mask_chemical, "subject_label_mapping"] = df_mapping.loc[mask_chemical, "subject_label_clean"].map(name_to_drugbank)



In [175]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping,subject_label_mapping,subject_label_clean
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN


#### 根据 label 映射修正 subject_mapping

In [176]:
# 只处理 subject_label_mapping 有值的行
mask_non_null = df_mapping["subject_label_mapping"].notna()

# 如果 mapping 不一致，用 label_mapping 替换
df_mapping.loc[mask_non_null & (df_mapping["subject_mapping"] != df_mapping["subject_label_mapping"]),
       "subject_mapping"] = df_mapping.loc[mask_non_null & (df_mapping["subject_mapping"] != df_mapping["subject_label_mapping"]),
                                   "subject_label_mapping"]


In [177]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping,subject_label_mapping,subject_label_clean
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,...,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,None,NaN


#### 对object做上述同样的映射

In [178]:
#初始化 object_mapping 列
df_mapping["object_mapping"] = df_mapping["object"]
#根据上文构建的 CHEBI → DrugBank 字典（基于 xrefs），应用映射到 object_mapping 列
df_mapping["object_mapping"] = df_mapping["object_mapping"].apply(
    lambda x: chebi_to_drugbank.get(x, x) if isinstance(x, str) and x.startswith("CHEBI:") else x
)
#基于 object_label 做进一步名称映射
# 初始化映射列
df_mapping["object_label_mapping"] = None

# 筛选 ChemicalEntity 行
mask_chem_obj = df_mapping["object_category"] == "biolink:ChemicalEntity"

# 标准化名称
df_mapping.loc[mask_chem_obj, "object_label_clean"] = df_mapping.loc[mask_chem_obj, "object_label"].str.strip().str.lower()

# 应用名称映射
df_mapping.loc[mask_chem_obj, "object_label_mapping"] = df_mapping.loc[mask_chem_obj, "object_label_clean"].map(name_to_drugbank)

# 替换 object_mapping（优先使用 label 映射值）
mask_non_null_obj = df_mapping["object_label_mapping"].notna()
df_mapping.loc[
    mask_non_null_obj & (df_mapping["object_mapping"] != df_mapping["object_label_mapping"]),
    "object_mapping"
] = df_mapping.loc[
    mask_non_null_obj & (df_mapping["object_mapping"] != df_mapping["object_label_mapping"]),
    "object_label_mapping"
]


In [179]:
#其他的ID映射
# 替换 NCBIGene: → ENTREZ:，适用于 subject_mapping 和 object_mapping
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].str.replace(r"^NCBIGene:", "ENTREZ:", regex=True)
df_mapping["object_mapping"] = df_mapping["object_mapping"].str.replace(r"^NCBIGene:", "ENTREZ:", regex=True)

# 替换 Reactome → REACT，适用于 subject_mapping 和 object_mapping
df_mapping["subject_mapping"] = df_mapping["subject_mapping"].str.replace(r"^Reactome:", "REACT:", regex=True)
df_mapping["object_mapping"] = df_mapping["object_mapping"].str.replace(r"^Reactome:", "REACT:", regex=True)

In [180]:
#删除多余的映射列
columns_to_drop = [
    "subject_label_clean",
    "subject_label_mapping",
    "object_label_clean",
    "object_label_mapping"
]
df_mapping.drop(columns=columns_to_drop, inplace=True, errors="ignore")
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,publications,has_evidence,primary_knowledge_source,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping,object_mapping
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,...,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32778
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,...,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32837
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,...,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32842
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,...,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32869
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,...,PMID:21267068|MGI:4888295,NaN,infores:mgi,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32870


#### 对subject_category和object_category进行格式化处理

In [181]:
# 拆分 biolink:前缀，生成新的列
df_mapping["source_type"] = df_mapping["subject_category"].str.replace("biolink:", "", regex=False)
df_mapping["target_type"] = df_mapping["object_category"].str.replace("biolink:", "", regex=False)

In [182]:
#替换 AnatomicalEntity → Anatomy, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("AnatomicalEntity", "Anatomy")
df_mapping["target_type"] = df_mapping["target_type"].replace("AnatomicalEntity", "Anatomy")

#替换 PhenotypicFeature → Phenotype, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("PhenotypicFeature", "Phenotype")
df_mapping["target_type"] = df_mapping["target_type"].replace("PhenotypicFeature", "Phenotype")

#替换 MolecularActivity → MolecularFunction, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("MolecularActivity", "MolecularFunction")
df_mapping["target_type"] = df_mapping["target_type"].replace("MolecularActivity", "MolecularFunction")

#替换 ChemicalEntity → Compound, 适用于source_type和target_type
df_mapping["source_type"] = df_mapping["source_type"].replace("ChemicalEntity", "Compound")
df_mapping["target_type"] = df_mapping["target_type"].replace("ChemicalEntity", "Compound")


# 创建新列 relation_type = predicate:source_type:target_type
df_mapping["relation_type"] = df_mapping["predicate"] + ":" + df_mapping["source_type"] + ":" + df_mapping["target_type"]

# 替换 biolink:predicate:source:target 为 biolink::predicate::source:target
df_mapping["relation_type"] = df_mapping["relation_type"].str.replace(
    r"^biolink:([^:]+):", r"biolink::\1::", regex=True
)

In [183]:
df_mapping.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping,object_mapping,source_type,target_type,relation_type
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32778,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32837,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32842,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32869,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32870,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy


#### 对relation_type进行标准化处理

In [184]:
df_mapping["relation_type"].unique()

array(['biolink::expressed_in::Gene:Anatomy',
       'biolink::expressed_in::Gene:Cell',
       'biolink::expressed_in::Gene:CellularComponent',
       'biolink::actively_involved_in::Gene:BiologicalProcess',
       'biolink::acts_upstream_of::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_positive_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_negative_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within_positive_effect::Gene:BiologicalProcess',
       'biolink::acts_upstream_of_or_within_negative_effect::Gene:BiologicalProcess',
       'biolink::actively_involved_in::Protein:BiologicalProcess',
       'biolink::acts_upstream_of_or_within::Protein:BiologicalProcess',
       'biolink::participates_in::Compound:Pathway',
       'biolink::contributes_to::Gene:Disease',
       'biolink::gene_associated_with_condition::Gene:Disease',
       'biolink::contributes_to:

In [185]:
df_mapping["relation_type"].nunique()

179

In [160]:
relation_counts_df = df_mapping["relation_type"].value_counts().reset_index()
relation_counts_df.columns = ["relation_type", "count"]
relation_counts_df.to_csv("relation_type_counts.csv", index=False)


In [186]:
import pandas as pd

# 1. 加载 Excel 文件中 sheet='relation_type' 的合法关系类型
relation_file_path = "/Users/zhuzhixing/KG/relation_types_20250524.xlsx"  # 替换为你的路径
relation_types_df = pd.read_excel(relation_file_path, sheet_name="relation_type")

# 2. 获取所有合法的 relation_type（去除缺失值）
valid_relations = relation_types_df["relation_type"].dropna().unique()

# 3. 筛选 df_mapping 中 relation_type 在合法列表中的行
filtered_df = df_mapping[df_mapping["relation_type"].isin(valid_relations)]

filtered_df.head()

,subject,subject_label,subject_category,subject_taxon,subject_taxon_label,negated,predicate,object,object_label,object_category,...,aggregator_knowledge_source,object_taxon,object_taxon_label,object_taxon_1,object_taxon_label_1,subject_mapping,object_mapping,source_type,target_type,relation_type
0,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32778,pretectum,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32778,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
1,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32837,telencephalon septum,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32837,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
2,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32842,neocortex,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32842,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
3,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32869,superior colliculus,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32869,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy
4,MGI:108061,Wnt10b,biolink:Gene,NCBITaxon:10090,Mus musculus,NaN,biolink:expressed_in,EMAPA:32870,inferior colliculus,biolink:AnatomicalEntity,...,infores:monarchinitiative|infores:alliancegenome,NaN,NaN,NaN,NaN,ENTREZ:22410,EMAPA:32870,Gene,Anatomy,biolink::expressed_in::Gene:Anatomy


In [187]:
len(filtered_df)

11846890

In [188]:
len(df_mapping)

12586584

In [189]:
formatted_df = pd.DataFrame()
formatted_df["source_name"] = df_mapping["subject_label"]
formatted_df["source_type"] = df_mapping["source_type"]
formatted_df["source_id"] = df_mapping["subject"]
formatted_df["target_name"] = df_mapping["object_label"]
formatted_df["target_type"] = df_mapping["target_type"]
formatted_df["target_id"] = df_mapping["object"]
formatted_df["relation_type"] = df_mapping["relation_type"]

formatted_df["resource"] = "biolink"

invalid_formatted_df = formatted_df[formatted_df["source_id"].isna() | formatted_df["target_id"].isna()]
invalid_formatted_df.to_csv("invalid_formatted_biolink.tsv", index=False, sep="\t")

formatted_df = formatted_df[~formatted_df["source_id"].isna() & ~formatted_df["target_id"].isna()]
formatted_df.to_csv("formatted_biolink.tsv", index=False, sep="\t")


In [190]:
formatted_df.head()

,source_name,source_type,source_id,target_name,target_type,target_id,relation_type,resource
0,Wnt10b,Gene,MGI:108061,pretectum,Anatomy,EMAPA:32778,biolink::expressed_in::Gene:Anatomy,biolink
1,Wnt10b,Gene,MGI:108061,telencephalon septum,Anatomy,EMAPA:32837,biolink::expressed_in::Gene:Anatomy,biolink
2,Wnt10b,Gene,MGI:108061,neocortex,Anatomy,EMAPA:32842,biolink::expressed_in::Gene:Anatomy,biolink
3,Wnt10b,Gene,MGI:108061,superior colliculus,Anatomy,EMAPA:32869,biolink::expressed_in::Gene:Anatomy,biolink
4,Wnt10b,Gene,MGI:108061,inferior colliculus,Anatomy,EMAPA:32870,biolink::expressed_in::Gene:Anatomy,biolink


In [191]:
len(formatted_df)

12586584

In [192]:
import os
import os.path as osp
import subprocess


def format_biolink(filename):
    def get_project_root():
        try:
            return osp.dirname(osp.dirname(os.getcwd()))
        except Exception as e:
            raise RuntimeError(f"Failed to determine project root: {e}")

    try:
        root_dir = get_project_root()
        print(f"Project root directory: {root_dir}")
    except RuntimeError as e:
        print(e)
        exit(1)

    database = "customdb"
    relations_path = osp.join(
        root_dir,
        "relations",
        "Monarch",
        filename,
    )
    output_dir = osp.join(
        root_dir, "formatted_relations", "monarch"
    )
    entities_path = osp.join(root_dir, "entities.tsv")
    log_file = osp.join(output_dir, "log.txt")
    relation_types_file = osp.join(root_dir, "relation_types.tsv")

    command = [
        "graph-builder",
        "--database",
        database,
        "-d",
        relations_path,
        "-o",
        output_dir,
        "-f",
        entities_path,
        "-n",
        "20",
        "--download",
        "--skip",
        "-l",
        log_file,
        "--debug",
        "--relation-type-dict-fpath",
        relation_types_file,
    ]

    print("Executing command:", " ".join(command))

    try:
        subprocess.run(command, check=True)
    except FileNotFoundError:
        print(
            "Error: 'graph-builder' command not found. Make sure it is installed and available in the PATH."
        )
        exit(1)
    except subprocess.CalledProcessError as e:
        print(f"Error: Command execution failed with return code {e.returncode}")
        print(f"Output: {e.output}")
        exit(1)
    except Exception as e:
        print(f"Unexpected error: {e}")
        exit(1)

In [ ]:
format_biolink("formatted_biolink.tsv")

Project root directory: /Users/zhuzhixing/KG/biomedgps-data/graph_data
Executing command: graph-builder --database customdb -d /Users/zhuzhixing/KG/biomedgps-data/graph_data/relations/Monarch/formatted_biolink.tsv -o /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch -f /Users/zhuzhixing/KG/biomedgps-data/graph_data/entities.tsv -n 20 --download --skip -l /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch/log.txt --debug --relation-type-dict-fpath /Users/zhuzhixing/KG/biomedgps-data/graph_data/relation_types.tsv


2025-05-25 08:10:52 - cli:171 - INFO - Run jobs with (output_dir: /Users/zhuzhixing/KG/biomedgps-data/graph_data/formatted_relations/monarch, db file/directory: /Users/zhuzhixing/KG/biomedgps-data/graph_data/relations/Monarch/formatted_biolink.tsv, databases: ('customdb',), download: True, skip: True)
2025-05-25 08:10:55 - base_parser:229 - INFO - Using allow_ignore_checking_errors=all to ignore the checking errors.
2025-05-25 08:11:14 - customdb_parser:104 - INFO - Get 12586584 relations
